# Old Faithful

In [ ]:
# Data file in this notebook is from https://www.stat.cmu.edu/~larry/all-of-statistics/=data/faithful.dat
# The original paper is available as https://tommasorigon.github.io/StatI/approfondimenti/Azzalini1990.pdf

In [ ]:
# Standard definitions and options
from datascience import Table    # high-level abstraction
import pandas as pd              # mid-level data frames and series
import numpy as np               # low-level arrays and vectors

import matplotlib                # plotting
matplotlib.use('Agg')                        # make nice screen plots
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')             # a particular plot format
plt.rcParams['figure.figsize'] = (10.0, 5.0) # wide plots to use space well

In [ ]:
# Read in the data from a CSV file - headers taken from file
data = Table.read_table("oldfaithful.csv")

In [ ]:
# Take a look at the data
data

In [ ]:
# Old Faithful is famous for its repeatability - lets check some statistics
data[2].mean()            # data[2] is the Interval column

In [ ]:
data['Interval'].std()    # but we can also refer to it by name

In [ ]:
data['Interval'].min()

In [ ]:
data['Interval'].max()     # all the usual summary statistics are available

In [ ]:
# While we're here, let's look at the other data we have
data['Duration'].mean(), data['Duration'].std()         # two statements on a line using commas

In [ ]:
# Let's see what the distribution looks like
plt.hist(data['Interval'], bins=30)
plt.figtext(0.75,0.5, data.to_df()['Interval'].describe().to_string())   # add descripitive text block from pandas
plt.title("Interval");                                                   # semicolon suppresses printing value

In [ ]:
# Not particularly Gaussian!
# Maybe there's two peaks there. But that still doesn't give us a better way to predict the eruption.
# Look at other information we have:
plt.hist(data['Duration'], bins=30)
plt.figtext(0.3,0.4, data.to_df()['Duration'].describe().to_string())
plt.title("Duration");

In [ ]:
# Maybe there's a correlation?
np.corrcoef(data['Duration'], data['Interval'])

In [ ]:
# that's pretty strong, let's look at it
plt.plot(data['Duration'], data['Interval']);

In [ ]:
# Maybe plotting as points would be better...
plt.plot(data['Duration'], data['Interval'],"ob");   # o: dots  b: blue

In [ ]:
# There seems to be two populations there!

# If we select just one:
long_duration_data = data.where(data['Duration'] > 3.2)
plt.hist(long_duration_data['Duration'], bins=20)
plt.figtext(0.1,0.5, long_duration_data.to_df()['Duration'].describe().to_string())
plt.title("Duration > 3.2");

In [ ]:
# But of course duration is more compact because we selected a narrower range,  How about interval?
plt.hist(long_duration_data['Interval'], bins=20)
plt.figtext(0.75,0.5, long_duration_data.to_df()['Interval'].describe().to_string())
plt.title("Interval with Duration > 3.2");

In [ ]:
# We're down to 50% in 8 minutes and an RMS of 6 minutes on a mean of 80; 10%!
#
# The shorter duration blob is left for the reader...

In [ ]:
# Try fitting a line instead using two populations
d = np.polyfit(data['Duration'], data['Interval'],1)  # 1 is the order of the polynomial; try 0 and 2
f = np.poly1d(d)
data['trendline'] = f(data['Duration'])   # adds a column with the line values

# To get a nice plot with 2nd order, we need to sort the data in the plot as it draws line between pairs
# To see this, you might want to try plotting without the sort.

temp = data.sort('Duration')

plt.plot(temp['Duration'], temp['Interval'],"ob");
plt.plot(temp['Duration'], temp['trendline'],"k");

In [ ]:
# See how wide the difference from the linear fit is
plt.hist(data['Interval']-data['trendline'], 30)
plt.figtext(0.75,0.5, (data.to_df()['Interval']-data.to_df()['trendline']).describe().to_string())
plt.title("Difference from Fit");

In [ ]:
# Is one of these better (less variance) than the other, or are they about the same?

In [ ]:
# Try plotting duration and wait time vs the event number. Is there a pattern there?

In [ ]:
plt.plot(data['N'], data['Interval'],"b");

In [ ]:
plt.plot(data['N'], data['Duration'],"b");

In [ ]:
# Do points or lines make the display clearer?

In [ ]:
# You can calculate the difference to the previous value and add it as a column:
data['deltaD'] = data.to_df()["Duration"].diff()  # pandas' diff() makes an address of row-by-row differences
data['deltaD'][0:5]                               # [0:5] selects out elements                        

In [ ]:
data['deltaI'] = data.to_df()["Interval"].diff()
data['deltaI'][0:5]                               # notation for just some elements

In [ ]:
# Plot deltaD vs deltaI and see if there's any grouping

In [ ]:
# Does that look like three groups?  Is there a way to understand this?

In [ ]:
plt.hist(data['deltaD'], 30);

In [ ]:
plt.plot(data['deltaD'], data['Interval'],"ob");

In [ ]:
... # what else can you plot to understand this?

In [ ]:
# Two of those groups look tightly clustered. But something goes wrong when Duration doesn't alternate...
# What data selection and plots would help you understand that?

In [ ]:
# We can split the data into three groups in deltaD
high_delta = data.where(data['deltaD'] >1.5)
mid_delta = data.where((data['deltaD'] >-1.5) & (data['deltaD'] < 1.5))  # and/or in selections
low_delta = ... # fill this in

In [ ]:
plt.hist(high_delta['Interval'], bins=15)
plt.figtext(0.75,0.5, high_delta.to_df()['Interval'].describe().to_string())
plt.title("Duration for high deltaD");

In [ ]:
plt.hist(mid_delta['Interval'], bins=15)
plt.figtext(0.1,0.5, mid_delta.to_df()['Interval'].describe().to_string())
plt.title("Duration for mid deltaD");

In [ ]:
# plot the low_delta sample, with description block and title


In [ ]:
# We can plot that central sample and see where the tail comes from
plt.plot(mid_delta['Duration'], mid_delta['Interval'],"ob");

In [ ]:
# We can plot all three groups on the same plot using color
plt.plot(high_delta['Duration'], high_delta['Interval'],"or");
plt.plot(mid_delta['Duration'], mid_delta['Interval'],"oy");
plt.plot(low_delta['Duration'], low_delta['Interval'],"ob");

In [ ]:
# So there seem to be three different physical things going on here
#
# But remember, you can't use deltaD as a predictor, because it involves Duration:  You don't know that yet
# To truely predict (as opposed to investigating the cause), you have to go back even further in time